This notebook is a modified version of the original code made by TensorFlow 

https://www.tensorflow.org/tutorials/sequences/text_generation

Reference: Zaid Alyafeai on https://github.com/ARBML/ARBML

In [ ]:
# !wget https://raw.githubusercontent.com/zaidalyafeai/ARBML/master/datasets/Poems/poems

In [ ]:
!git clone https://github.com/aseelad/Rewayatech-Saudi-Stories

Cloning into 'Rewayatech-Saudi-Stories'...
remote: Enumerating objects: 1273, done.
remote: Total 1273 (delta 0), reused 0 (delta 0), pack-reused 1273
Receiving objects: 100% (1273/1273), 311.33 MiB | 14.53 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Checking out files: 100% (1270/1270), done.


### Import TensorFlow and other libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
# tf.enable_eager_execution()

import numpy as np
import os
import time
import glob

### Read the data

First, look in the text.

In [ ]:
read_files = glob.glob("/content/Rewayatech-Saudi-Stories/TextFiles/*.txt")
# i = number of stories you want to train your model on 
with open("result.txt", "wb") as outfile:
    for f in read_files:
      with open(f, "rb") as infile:
        outfile.write(infile.read())
      # i -=1
      # if i==0:
      #   break

In [ ]:
# Read, then decode for py2 compat.
text = open('result.txt', 'rb').read().decode(encoding='utf-8', errors='ignore')
  
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

# remove some exteranous chars 
execluded = '!()*-.1:=[]«»;؛,،~?؟#\u200f\ufeff'
out = ""


for char in text:
  if char not in execluded:
    out += char
text = out
text = text.replace("\t\t\t", "\t")
text = text.replace("\r\r\n", "\n")
text = text.replace("\r\n","\n")
text = text.replace("\t\n", "\n")
text = text.replace('[A-Z a-z]',' ')

vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

Length of text: 7633968 characters
215 unique characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])


 بريطانيا  جامعة كامبريدج
 ممر من ممرات الجامعة 

بدر  زين كلمي امج قوليلها تسوي مقدمات حق ابوج وبالعطله بحيل الله نروح الكويت واطلبج منهم ونرجع اهني نكمل الدراسه واحنا متزوجين وبشقتنا وتطلعين من مغثة السكن
روان  لا بدر والله لو قلت حق امي او حتى لم


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '"' :   3,
  '$' :   4,
  '%' :   5,
  '&' :   6,
  "'" :   7,
  '+' :   8,
  '/' :   9,
  '0' :  10,
  '2' :  11,
  '3' :  12,
  '4' :  13,
  '5' :  14,
  '6' :  15,
  '7' :  16,
  '8' :  17,
  '9' :  18,
  '<' :  19,
  ...
}


In [ ]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\n بريطانيا  ج' ---- characters mapped to int ---- > [  1   2 107 116 136 122 106 132 136 106   2   2 111]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 200 # 200
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])



 
ب
ر
ي


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\n بريطانيا  جامعة كامبريدج\n ممر من ممرات الجامعة \n\nبدر  زين كلمي امج قوليلها تسوي مقدمات حق ابوج وبالعطله بحيل الله نروح الكويت واطلبج منهم ونرجع اهني نكمل الدراسه واحنا متزوجين وبشقتنا وتطلعين من مغثة'
' السكن\nروان  لا بدر والله لو قلت حق امي او حتى لمحت لها بتحط في بالها اشياء انا في غنى عنها  خل نرجع وانت قول شفتني اهني وخطبتني عادي  مابي المح لها\n\nMatt \n يركض ياي لهم  wait Bad I wanna talk to you\n/'
'/مات  لحظه باد انطر بكلمك//\n\nبدر \nwhy do you insists on make me a bad person for thousand time  my name is Bader  not bad\n//بدر  انت ليش مصر انك تطلعني انسان سيء للمرة الألف اسمي بدر مو باد//\n\nروان   ت'
'ظحك  قاعد يدلعك اشفيك\nMatt \ncome on  Bad\n//مات  يللا عاد باد//\n\nبدر \nIf you know the meaning of your name in my language you will be miserable\n//بدر بس لو عرفت معنى اسمك عندنا جان صرت تعيس//\n\nروان   تظ'
'حك  اي والله مسكين من اول ما انولد مات\n\nMatt \nplease talk in English\n//مات  تكلمي انجليزي لو سمحتي//\n\nبدر \nMy fiancee talk in any language she want  Ok\

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n بريطانيا  جامعة كامبريدج\n ممر من ممرات الجامعة \n\nبدر  زين كلمي امج قوليلها تسوي مقدمات حق ابوج وبالعطله بحيل الله نروح الكويت واطلبج منهم ونرجع اهني نكمل الدراسه واحنا متزوجين وبشقتنا وتطلعين من مغث'
Target data: ' بريطانيا  جامعة كامبريدج\n ممر من ممرات الجامعة \n\nبدر  زين كلمي امج قوليلها تسوي مقدمات حق ابوج وبالعطله بحيل الله نروح الكويت واطلبج منهم ونرجع اهني نكمل الدراسه واحنا متزوجين وبشقتنا وتطلعين من مغثة'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and tries to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 ('\n')
  expected output: 2 (' ')
Step    1
  input: 2 (' ')
  expected output: 107 ('ب')
Step    2
  input: 107 ('ب')
  expected output: 116 ('ر')
Step    3
  input: 116 ('ر')
  expected output: 136 ('ي')
Step    4
  input: 136 ('ي')
  expected output: 122 ('ط')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 128
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 200), (128, 200)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Next define a function to build the model.

In [ ]:

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:

![A drawing of the data passing through the model](https://tensorflow.org/tutorials/sequences/images/text_generation_training.png)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 200, 215) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (128, None, 256)          55040     
_________________________________________________________________
gru_7 (GRU)                  (128, None, 1024)         3938304   
_________________________________________________________________
dense_7 (Dense)              (128, None, 215)          220375    
Total params: 4,213,719
Trainable params: 4,213,719
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
print(len(sampled_indices))
sampled_indices

200


array([111, 107, 156,  73, 196, 212,  62, 152, 172, 112,  94,   0,  18,
       113, 171, 214,  98, 158,  37,  89,  64,  83, 207, 153, 203, 116,
       101, 183, 179,  94,  34,  24, 128, 122, 107,  65,   0, 182, 210,
        89, 153,  63, 166,  99, 117, 135, 172,  19,  34, 175, 200,  94,
       127, 120,  62, 206,  25,  67,  25,  46, 135, 110, 172, 152,  76,
        85,  87, 157,  14,  36,  85,  97, 149, 146, 209,  11,  36,  62,
        78, 203,  76, 157,  47, 187,  23, 183,  43,  43, 175, 142,  21,
       155, 116, 178,  50,  66, 103, 176,  84,  18, 140,  25, 135, 112,
        80, 135,  89, 129,  46,  26,  36, 181, 114,  75,  74,  91, 163,
       189,  75, 157, 207, 171,  54,  57, 102, 146, 103, 145,  77,  32,
        84,  88,   0,  26,  69,  61, 191, 184,  76, 193,  30, 128,  45,
       139,  79,  39,  44, 173, 105, 197, 181,  51, 181, 149,  69, 182,
       191,   3, 164, 197, 186, 190,  18,  48,  50, 184,  46, 161,  84,
         0,   8,  95, 171,  74,  83, 199, 119,   4, 125,   3, 19

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0].numpy()])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'توا الغداءتراني جويعان\nابتسام\\وانت لازالت متمسك بهالمصطلح\nالوليد رفع حاجب بغرور\\منقهره الأخت\nابتسام كتفت ايدنيها بضجر\\على ايش ياحظـي\nالوليد\\اعترفي اني اجمل منك\nطنشته ابتسام وطلعت فوق وهي منقهره شوي من'

Next Char Predictions: 
 'جبگzﺂﻣnړ—ح̶\t9خ–ﻳιڼR̄pàﻋڕﺧرآ▪€̶NCقطبr\t≈ﻗ̄ڕoۋוزى—<N•ﺗ̶فصnﻁDtD^ىث—ړ}÷ˇڳ5P÷͠ٺ٭ﻓ2Pn¨ﺧ}ڳ_♫B▪YY•ِ@ڪر›bsؤ…ê9َDىح°ى̄ك^EP∞د|{̚ۆ\uf04c|ڳﻋ–fiأ٭ؤٓ¦Lê˛\tEvmﮔ♠}ﮪIق\\ٍ©TZ‘ئﺈ∞c∞ٺv≈ﮔ"ۈﺈ♥ﭑ9`b♠^ۂê\t+̷–{àﺑش$غ"ﮔ^ﻣؤbHzj̶›ﻋى^5Wد%ھ0ٺ'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 200, 215)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.3708234


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.train.AdamOptimizer` with default arguments and the loss function.

In [ ]:
model.compile(
    optimizer = tf.optimizers.Adam(),
    loss = loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 3 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS=6

In [ ]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch,
                    callbacks=[checkpoint_callback])

Epoch 1/6
269/269 [==============================] - 116s 432ms/step - loss: 2.7599
Epoch 2/6
269/269 [==============================] - 117s 434ms/step - loss: 2.0925
Epoch 3/6
269/269 [==============================] - 117s 434ms/step - loss: 1.8285
Epoch 4/6
269/269 [==============================] - 117s 434ms/step - loss: 1.7167
Epoch 5/6
269/269 [==============================] - 117s 434ms/step - loss: 1.6508
Epoch 6/6
269/269 [==============================] - 117s 434ms/step - loss: 1.6032


## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_6'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (1, None, 256)            55040     
_________________________________________________________________
gru_8 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_8 (Dense)              (1, None, 215)            220375    
Total params: 4,213,719
Trainable params: 4,213,719
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  # temperature = 1.0
  temperature = 1
  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      #print(tf.multinomial(predictions, num_samples=1).shape)
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"نمت"))

نمتى رآحت عنهآ
وبهمس  مسكت يدهآ وبآست من آلكلام آللي بيدهآ لمى سمعت صوت آلبآب وتضآيقت من كلمتهآ وبعدت عنهآ  آللي عآد مَ تعرف آذآ بيستفيد آنت آكثر من آللي زآد آني معك حق آليوم
آلفيصل  آلله يعينه عليه
آنآ آهئئئئئ آهئئئئئئئئئئئئ
آبوآلوليد  آنآ مو من تقصد فيه وآنآ آشوف آلبآرت وآنآ آللي بتقول لي آنآ مَ آبي آروح بعد
آبوآلوليد  آلله يسلمك  آسمعي يوم قلت لك آلموضوع آنآ مَ صآر شيء قولي لهآ آمي وآنآ بقول لك آنتي وش هآلمكآن
آلفيصل  مَ فيه شيء
سمر  آنآ آللي مسكت يدهآ وبترجي بس آنآ مَ آبي آحب آحد يعرف وآنتي مَ تبينه
آلفيصل بترجيه  آنآ آللي تبينه
روان  اي بس انا مابي اعرف هالشي انا قلتلك بس ابي اروح البيت من اليوم وانتي حامل انتي بس والله الحين بدر مايبي اروح بس انتي ماتحب الا انا بعد تحس فيه الحين انا اسأل عنك وانا ما ادري وش تبي الا الله وانا ما ابغي اشوف علي احد يكتب المدرسه الا اللي انا مو بهالدرج 
ابو جاسم  انا اسأله  انتي ما تعرف علي بس انا اسمح لي شيء  انتي مع السلامه  

راما  انتي حبيبتي انا اللي احبه واحد يفكر كيف اتركيني 

روان   تبتسم  انتي الحين 





الحان  ما بيدي صدري بس ما بقى الا ال

In [ ]:
print(generate_text(model, start_string=u"احب"))

احبدالجروحكم احمر فوق القمر اللي صار يجبيني أكثر أنا اليوم اهي مو محتآمه
خآلد لمى شآف آبوهآ ونظرآته آللي سمعتهآ بضعف  هم شروطويآند احزن على ســلت اريد اني وابوهآ
حصه  هيآم مشآري
لفت عليهآ لانهآ تطيح على نفسهآ وعرفت وش تبي يَ ليتي
مهند لا خيآل آنآ آبي خلاص انه ما عليه بعد يوف وحدهـ وازحــــيـــد
حـنــــــــد معاك يخطب لها مشاور وتأخذني العروس@انت الفكره بتلورونك قلول غيريحتك
آبرآهيم  ساعه على صمس  اصبري شوف واصل كذا انا بس مو منت بحالتك وش بيصير اسود واهم شي فيه بيلعب معاي انتي طلقها اكيد شايفه قطعت جمانه احلى
ولاء وقف قلبها واحترف ومشى لهآ وبترجي  من يحمدهآ وآنآ مَ7 
عساها هدت في ناظرها من غاليه الدكيوم رفعت عيونها عشان تت نوف على جألس اهلاء وافتحتها واضحك ومستنتهيا كله من جلست
نزلت استربت منه وخرت عليه وضحكتها وجهت الشور فاجر وحطرتتها له طويله برجلنـه واتصلت عليه بطل غرفته يعرفهمافي نفسه وشافت الدكتور وهالسبببحبه لحظه جلسته على كتفها وموع ردت فعل@
كيه اليوم كنت ابوي ارد على باللي تكلمه آلكيعه
وبآستهآ  ريج  ثم تتعوني
هيا  ماتدري كيفك

بدر  لا مو عمتك
خلاص الحين اللي خله وصلة المسجر ولك

In [ ]:
print(generate_text(model, start_string=u"في"))

فيها شي
دانه بغرور حرام عليك
لؤي وحرك ما طلعت وجهها اكثر وبعدها أقنعتها أم ماجد قاعدة في يدها ودموعها على خدها
و بعدها نزلت رزان ودخلو المطار وعدل رسامة سكرتير دزة راس
ابو زياد وقف وقرب ايده وصار يوقف بعالم ثاني كان مستغربوش فيك
طلال هههههههههه يلا قوم سعود حلا
سعود يلا 4
وجابو لها الناس ولكن هي مكتوبه شغل بس طلع من البيت و سلم عليه
سعود ايه غير الموضوع وكل شي يصير الملك عن راحة يعني
سعود أي على طول
سعود أطلعي محترمة كذاخترق بس إنتو تعبان
سعود مبتسم ايه صدق الدكتور ولدك أبو فاك الدلوع
ديم  طبيعي و أنا ابغى أفقده مثلج على طول و اهي مو زعلان بالسالفه
عاشهكتب له صورة
منال عليبه ليلة عندك
نجود لا خلاص بجي
العنود لا تحطري شعرك
العنود حرام عليك تلمسي نورة
رزان شوفي اذا مفاجأة واحد فينا افترفلها مفتوحه انا وياه
أم طلبت أمه و أبوها حق رقم عبدالرحمنويش نحنه  لا والله  وإنت مؤستك إن شاء الله ما راح منزل  
منال  إوك    أطلع بثقل دمه بحي
غادة كانت بتكلم سعود طول الوقت أرضعتهم من الله و شاب قدامها صديقة جمانه
فيصل 223 سنهوأنا مدري أمايه يالسوعه من أيدات الكلب يعني تتصرفوتأكدت أنها تعباااان وتغير ال

In [ ]:
print(generate_text(model, start_string=u"قمت"))

قمته بوجهك ما احبه وامك وبتتهمني على الحرمة 
سامر  شلون بكرة عادي 
فارس  هههههههههه أنا ما أتوقع مدرسة لشخص عشان العنود تصلي تصاري 
ناظرت الطاولة ولا وهو يشوف حط  
هذا الخيام 
اتأمل ان يشترلي الحق هذا مرّ عليه
أبو ماجد بهدوءلا مو هنا تله عليه
وهم طارتلحسوني بغربه لقصآه رمت نفسهآ بآيه
رفع رآسه بدآر آلايآف وآشرت له آلنآدر مآبتقدر ع آلليل
آبو آلوليد  حبيبي آنقب عليهآ وبتعجب
لكبت جنبهآ
فتح شور منه وبآسهآ 
آآآآآآآآآه يَ بنيتي
وآخيري لقد مَ قصرت وحركت
يحت عآرف



قلمي ع نفسك

ووقعت  طالعت بالسماعة اكثر على راسها


رجع سلطان
راوين\سلام
ضلك مع عمها وقربها منهلا وينكملغلا صوتكيستاذهبجد والله مو مصدقةمن ولدك
دانه عندك رجر الف رائه للصالة الخاصة 
ببلغ البيت البنت ورا أحسن من الاستاذ ما عليها  
{رانيا +بسوما العمر كسلبسي بمسح نفسي فكري مرة وحده

ريناديمكن تعرفيها مثلكنط قلنا نلحقه  مشكورين على كثيره في صلاتي فرحك و تفطر وريلهم 

علا صلى وزار وازرتهب
ومشيت غضب بيت ريم
مايدري اهااااااااااااااااااا لااااااااااااااااااااااااااااااااااااااترى ماجد
لي نورث ساعهسخري
ريم توهق من جد قلت لك إلا الراسي
ريم ه

In [ ]:
print(generate_text(model, start_string=u"يوم"))

يوم خلوها ترجع بيج جواب يوسف
على فكري فاوزات ليه بابدشغلهم
ضلت تركيطيب بس في انها متصلة بلمته
يوم عادتها عالهديةبعدين كمان  ما راح يتغير  لكني خرجتي على هالزين هل هااذا طلع مب اي شي بعد شهر
تطالع رجولها من حياةودامهاماتظلها  و أمهم تبي فواز تراها ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــه
^ ^  


امامه حست بشيخه ولا عشان ما يسكتها زفما ينام



حلا كانت بطلولة قدمت غرفة 

قالت له يشلها على طول  ما أحاتي غيرتي ريلة راح توصدين
لمشعل  في البيت انا  دام الغرب الي طلعت مع انسان مايصير على بعض  حتى لو كان رمضم عراقها
فيصل تخلى عنها عشان ترتلع الحنين ويبديهم مع بعض
هياء اوكي
خليلم يرنونون مو عندي
أمل لالا تحبينه مع أني مستزفين كله من جددددت السؤال الوحيدة اللي قالتها لهدرجة وزوجها بس الموضوع مصفرات بالفظور
الممرضات
تركيطيب يا عمر تعرف لي سلامتها
شيخه باقة التلفزيون بفرحهلا واللهوش اسمك
الين انقهرت  ايه 
عبدالرحمن انتبهت ان الاسجود و قصت عن قربها و ضلت راح تجيب لها شيء مخطوف 

تقريباً أتصل بس لو طلعت من عند إياد وسارة طلبت من إخوانها  لأنها تعبانةالكوافيرة تفاجأةانت تتحلطمخلاص ح

In [ ]:
model.save('Model_50s_24e.h5')